In [10]:
import tensorflow as tf
import zipfile
import os
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.image as mpimg
from keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report, confusion_matrix
from keras.preprocessing import image
from keras.callbacks import ReduceLROnPlateau

In [11]:
tf.config.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [12]:
base_dir = 'dataset'
train_dir = os.path.join(base_dir, 'train')
test_dir = os.path.join(base_dir, 'test')

In [13]:
train_datagen = ImageDataGenerator(
                rescale=1./255,
                rotation_range=20,
                horizontal_flip=True,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                vertical_flip=True,
                fill_mode='nearest',
                validation_split=0.3)

validation_datagen = ImageDataGenerator(rescale = 1.0/255,
                                        validation_split=0.4)

train_generator = train_datagen.flow_from_directory(
        train_dir,  
        target_size=(224,224),
        batch_size=5,
        class_mode='categorical',
        subset='training')

validation_generator = validation_datagen.flow_from_directory(
        test_dir, 
        target_size=(224,224), 
        batch_size=4,
        class_mode='categorical',
        subset='validation')

Found 1313 images belonging to 4 classes.
Found 228 images belonging to 4 classes.


In [14]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(input_shape=(224,224,3),filters=64,kernel_size=(3,3),padding="same", activation="relu"),
    tf.keras.layers.Conv2D(filters=64,kernel_size=(3,3),padding="same", activation="relu"),
    tf.keras.layers.MaxPool2D(pool_size=(2,2),strides=(2,2)),
    tf.keras.layers.Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"),
    tf.keras.layers.Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"),
    tf.keras.layers.MaxPool2D(pool_size=(2,2),strides=(2,2)),
    tf.keras.layers.Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"),
    tf.keras.layers.Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"),
    tf.keras.layers.Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"),
    tf.keras.layers.MaxPool2D(pool_size=(2,2),strides=(2,2)),

    tf.keras.layers.Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"),
    tf.keras.layers.Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"),
    tf.keras.layers.Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"),
    tf.keras.layers.MaxPool2D(pool_size=(2,2),strides=(2,2)),

    tf.keras.layers.Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"),
    tf.keras.layers.Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"),
    tf.keras.layers.Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"),
    tf.keras.layers.MaxPool2D(pool_size=(2,2),strides=(2,2)),

    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(units=4096,activation="relu"),
    tf.keras.layers.Dense(units=4096,activation="relu"),
    tf.keras.layers.Dense(units=4, activation="softmax"),
])
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_26 (Conv2D)          (None, 224, 224, 64)      1792      
                                                                 
 conv2d_27 (Conv2D)          (None, 224, 224, 64)      36928     
                                                                 
 max_pooling2d_10 (MaxPoolin  (None, 112, 112, 64)     0         
 g2D)                                                            
                                                                 
 conv2d_28 (Conv2D)          (None, 112, 112, 128)     73856     
                                                                 
 conv2d_29 (Conv2D)          (None, 112, 112, 128)     147584    
                                                                 
 max_pooling2d_11 (MaxPoolin  (None, 56, 56, 128)      0         
 g2D)                                                 

In [15]:
learning_rate_reduction = ReduceLROnPlateau(monitor='val_accuracy',
                                            patience=2,
                                            verbose=1,
                                            factor=0.5,
                                            min_lr=0.000003)
                                            
model.compile(loss='categorical_crossentropy',
                optimizer='adam',
                metrics=['accuracy'])

history = model.fit(
    train_generator,
    epochs=10,
    validation_data=validation_generator,
    callbacks=[learning_rate_reduction])

Epoch 1/10


2022-06-21 20:46:59.770773: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


263/263 [==============================] - ETA: 0s - loss: 1.4219 - accuracy: 0.4029

2022-06-21 20:49:14.483445: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


263/263 [==============================] - 141s 533ms/step - loss: 1.4219 - accuracy: 0.4029 - val_loss: 1.4319 - val_accuracy: 0.2500 - lr: 0.0010
Epoch 2/10
263/263 [==============================] - 142s 537ms/step - loss: 1.3304 - accuracy: 0.4037 - val_loss: 1.4119 - val_accuracy: 0.2500 - lr: 0.0010
Epoch 3/10
263/263 [==============================] - ETA: 0s - loss: 1.3338 - accuracy: 0.4037
Epoch 3: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
263/263 [==============================] - 145s 551ms/step - loss: 1.3338 - accuracy: 0.4037 - val_loss: 1.4282 - val_accuracy: 0.2500 - lr: 0.0010
Epoch 4/10
 74/263 [=======>......................] - ETA: 1:40 - loss: 1.3261 - accuracy: 0.4076

KeyboardInterrupt: 

In [ ]:
def eval_plot(history):
    
    plt.figure(figsize=(14, 5))

    # Accuracy plot
    plt.subplot(1, 2, 1)
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    epochs = range(len(acc))
    acc_plot, = plt.plot(epochs, acc, 'r')
    val_acc_plot, = plt.plot(epochs, val_acc, 'b')
    plt.title('Training and Validation Accuracy')
    plt.legend([acc_plot, val_acc_plot], ['Training Accuracy', 'Validation Accuracy'])

    # Loss plot
    plt.subplot(1, 2, 2)
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    epochs = range(len(loss))
    loss_plot, = plt.plot(epochs, loss, 'r')
    val_loss_plot, = plt.plot(epochs, val_loss, 'b')
    plt.title('Training and Validation Loss')
    plt.legend([loss_plot, val_loss_plot], ['Training Loss', 'Validation Loss'])

def evaluate(model):
    
    validation_generator = train_datagen.flow_from_directory(
            test_dir, 
            target_size=(256,256), 
            batch_size=10, 
            class_mode='categorical',
            shuffle = False,
            subset='validation')

    batch_size = 10
    num_of_test_samples = len(validation_generator.filenames)

    Y_pred = model.predict_generator(validation_generator, num_of_test_samples // batch_size+1)
    y_pred = np.argmax(Y_pred, axis=1)

    print('\nConfusion Matrix\n')
    print(confusion_matrix(validation_generator.classes, y_pred))


    print('\n\nClassification Report\n')
    target_names = ['GINA', 'MATAHARI', 'SIRAT','ULI']
    print(classification_report(validation_generator.classes, y_pred, target_names=target_names))

def predict_image(image_upload, model = model):
    im = image_upload
    im_array = np.asarray(im)
    im_array = im_array*(1/225)
    im_input = tf.reshape(im_array, shape = [1, 256, 256, 3])

    predict_proba = sorted(model.predict(im_input)[0])[2]
    predict_class = np.argmax(model.predict(im_input))

    print('\n')
    plt.show()
    print("\nImage prediction result: ", predict_label)
    print("Probability: ", round(predict_proba*100,2), "%")
    print('\n')

In [ ]:
eval_plot(history)
evaluate(model)
